In [104]:
%pip install requests
%pip install attrs
%pip install ratelimit
%pip install pandas
%pip install python-dotenv

import pickle
import requests
import attr
import pandas as pd

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because norm

In [107]:
from dotenv import dotenv_values
env = dotenv_values("fronius.env")

ACCESSKEYID = env['ACCESSKEYID']
ACCESSKEYVALUE = env['ACCESSKEYVALUE']
ENDPOINT = "https://api.solarweb.com/swqapi"

headers = {
    'AccessKeyId': ACCESSKEYID,
    'AccessKeyValue': ACCESSKEYVALUE,
}

In [108]:
pvsystems = pd.DataFrame(
    requests.get(f'{ENDPOINT}/pvsystems', headers=headers).json()['pvSystems']
)
pvsystems.head()

,pvSystemId,name,address,pictureURL,peakPower,installationDate,lastImport,meteoData,timeZone
0,7e7aa106-19ea-4f79-ac69-e42883668ec7,Beniata derecha,"{'country': 'ES', 'zipCode': '03804', 'street'...",https://www.solarweb.com/Image/Show?pvSystemId...,10000.0,2018-01-01T00:00:00Z,2021-06-20T16:03:37Z,None,Europe/Paris
1,2e232019-c3b1-441c-b0ba-6c5050e8659e,EIMA Jesuset del Miracle,"{'country': 'ES', 'zipCode': '03803', 'street'...",https://www.solarweb.com/Image/Show?pvSystemId...,0.0,2000-01-01T00:00:00Z,2021-06-20T16:11:00Z,None,Europe/Paris
2,5633fc26-1704-4e81-8cbd-419d0bdac49b,Cotes Baixes Norte,"{'country': 'ES', 'zipCode': '03801', 'street'...",https://www.solarweb.com/Image/Show?pvSystemId...,8100.0,2018-03-01T00:00:00Z,2021-06-20T16:11:51Z,None,Europe/Paris
3,0b5a4728-5bd4-4190-a68d-ea1c01c8ede9,Tomas Llacer,"{'country': 'ES', 'zipCode': '03801', 'street'...",https://www.solarweb.com/Image/Show?pvSystemId...,10000.0,2000-01-01T00:00:00Z,2021-06-20T16:05:34Z,None,Europe/Paris
4,3bb39e2b-442e-4f27-ab07-fbfd5d4e363c,Polideportivo Francisco laporta,"{'country': 'ES', 'zipCode': '03801', 'street'...",https://www.solarweb.com/Image/Show?pvSystemId...,99000.0,2000-01-01T00:00:00Z,2021-06-20T16:21:33Z,None,Europe/Paris


In [26]:
frozenset(pvsystems.name)

frozenset({'Beniata derecha',
           'Beniata izquierda',
           'Biblioteca zona norte',
           'CEIP ROMERAL',
           'Cervantes CCJ',
           'Cotes Baixes Norte',
           'Cotes Baixes Sur',
           'EIMA Jesuset del Miracle',
           'Eduardo Latorre',
           'Escoleta Zona Norte',
           'Huerta Mayor',
           'Miguel Hernandez',
           'Polideportivo Francisco laporta',
           'San Vicente',
           'Santiago Paya Izquierda',
           'Santiago Payá Derecha',
           'Tomas Llacer'})

In [27]:
pvsystems.columns

Index(['pvSystemId', 'name', 'address', 'pictureURL', 'peakPower',
       'installationDate', 'lastImport', 'meteoData', 'timeZone'],
      dtype='object')

In [28]:
devices = {
    pvid: pd.DataFrame(
        requests
        .get(f'{ENDPOINT}/pvsystems/{pvid}/devices', headers=headers)
        .json()['devices']
    )
    for pvid in pvsystems.pvSystemId
}
next(iter(devices.values())).head()

,deviceType,deviceId,deviceName,deviceManufacturer,serialNumber,deviceTypeDetails,dataloggerId,nodeType,numberMPPTrackers,numberPhases,peakPower,nominalAcPower,firmware,isActive,activationDate,deactivationDate,isOnline,ipAddressV4
0,Inverter,bb177bc0-ce82-4c49-baa1-17b30626d56b,Beniata derecha Symo 10.0-3-M,Fronius,28402358,Fronius Symo 10.0-3-M,240.380967,97.0,2.0,3.0,"{'dc1': 10000.0, 'dc2': None}",10000.0,"{'updateAvailable': False, 'installedVersion':...",True,2018-01-10T00:00:00Z,None,NaN,NaN
1,Datalogger,2e303432-3833-3930-3637-000000000000,Datalogger,Fronius,NaN,NaN,240.380967,NaN,NaN,NaN,NaN,NaN,"{'updateAvailable': False, 'installedVersion':...",True,2000-01-01T00:00:00Z,None,True,194.224.148.153


In [97]:
len(pvsystems)

17

In [96]:
types_per_pv = dict()
for pvid, devs in devices.items():
    print("pvid: ", pvsystems[pvsystems['pvSystemId']==pvid].name)
    print(devs[['deviceType']].value_counts())
    for index, count in devs[['deviceType']].value_counts().iteritems():
        types_per_pv[index] = types_per_pv.get(index, 0) + count

print(types_per_pv)

pvid:  0    Beniata derecha
Name: name, dtype: object
deviceType
Datalogger    1
Inverter      1
dtype: int64
pvid:  1    EIMA Jesuset del Miracle
Name: name, dtype: object
deviceType
Datalogger    1
Inverter      1
SmartMeter    1
dtype: int64
pvid:  2    Cotes Baixes Norte
Name: name, dtype: object
deviceType
Inverter      3
Datalogger    1
dtype: int64
pvid:  3    Tomas Llacer
Name: name, dtype: object
deviceType
SmartMeter    2
Datalogger    1
Inverter      1
dtype: int64
pvid:  4    Polideportivo Francisco laporta
Name: name, dtype: object
deviceType
Inverter      3
Datalogger    1
dtype: int64
pvid:  5    Santiago Payá Derecha
Name: name, dtype: object
deviceType
Datalogger    1
Inverter      1
dtype: int64
pvid:  6    Beniata izquierda
Name: name, dtype: object
deviceType
Datalogger    1
Inverter      1
dtype: int64
pvid:  7    San Vicente
Name: name, dtype: object
deviceType
Datalogger    1
Inverter      1
SmartMeter    1
dtype: int64
pvid:  8    Eduardo Latorre
Name: name, dty

In [30]:
dtypes = {}
for pvid, device in devices.items():
    for dtype in device.deviceType:
        if not (dtype in dtypes):
            dtypes[dtype] = set()
        dtypes[dtype].update(device[device['deviceType']==dtype].dropna(axis=1).columns)

dtypes

{'Inverter': {'activationDate',
  'dataloggerId',
  'deviceId',
  'deviceManufacturer',
  'deviceName',
  'deviceType',
  'deviceTypeDetails',
  'firmware',
  'isActive',
  'nodeType',
  'nominalAcPower',
  'numberMPPTrackers',
  'numberPhases',
  'peakPower',
  'serialNumber'},
 'Datalogger': {'activationDate',
  'dataloggerId',
  'deviceId',
  'deviceManufacturer',
  'deviceName',
  'deviceType',
  'firmware',
  'ipAddressV4',
  'isActive',
  'isOnline'},
 'SmartMeter': {'activationDate',
  'deviceId',
  'deviceManufacturer',
  'deviceName',
  'deviceType',
  'firmware',
  'isActive'}}

In [48]:
flowdata = {
    pvid: requests.get(f'{ENDPOINT}/pvsystems/{pvid}/flowdata', headers=headers).json()
    for pvid in pvsystems.pvSystemId
}
next(iter(flowdata.values()))

{'pvSystemId': '7e7aa106-19ea-4f79-ac69-e42883668ec7',
 'status': {'isOnline': True},
 'data': {'logDateTime': '2021-06-20T09:25:49Z',
  'channels': [{'channelName': 'PowerFeedIn',
    'channelType': 'Power',
    'unit': 'W',
    'value': None},
   {'channelName': 'PowerLoad',
    'channelType': 'Power',
    'unit': 'W',
    'value': None},
   {'channelName': 'PowerBattCharge',
    'channelType': 'Power',
    'unit': 'W',
    'value': None},
   {'channelName': 'PowerPV',
    'channelType': 'Power',
    'unit': 'W',
    'value': None},
   {'channelName': 'PowerOutput',
    'channelType': 'Power',
    'unit': 'W',
    'value': 0.0},
   {'channelName': 'BattSOC',
    'channelType': 'Percentage',
    'unit': '%',
    'value': None},
   {'channelName': 'RateSelfConsumption',
    'channelType': 'Percentage',
    'unit': '%',
    'value': None},
   {'channelName': 'RateSelfSufficiency',
    'channelType': 'Percentage',
    'unit': '%',
    'value': None}]}}

In [32]:
channels = set()
for pvid, flow in flowdata.items():
    if flow['data']:
        for channel in flow['data'].get('channels', []):
            channels.add(channel['channelName'])

channels

{'BattSOC',
 'PowerBattCharge',
 'PowerFeedIn',
 'PowerLoad',
 'PowerOutput',
 'PowerPV',
 'RateSelfConsumption',
 'RateSelfSufficiency'}

In [22]:
from datetime import datetime, timedelta
now = datetime.utcnow().replace(microsecond=0)
bef = now - timedelta(days=1)
aggrdata = {
    pvid: requests.get(f'{ENDPOINT}/pvsystems/{pvid}/aggrdata', headers=headers, params={
        'from': bef.strftime("%Y-%m-%d"),
        'duration': 1
    }).json()
    for pvid in pvsystems.pvSystemId
}
next(iter(aggrdata.values()))

{'pvSystemId': '7e7aa106-19ea-4f79-ac69-e42883668ec7',
 'data': [{'logDateTime': '2021-06-18',
   'channels': [{'channelName': 'EnergyOutput',
     'channelType': 'Energy',
     'unit': 'Wh',
     'value': 1407.9019},
    {'channelName': 'EnergyDirectConsumption',
     'channelType': 'Energy',
     'unit': 'Wh',
     'value': 0.0},
    {'channelName': 'EnergyProductionTotal',
     'channelType': 'Energy',
     'unit': 'Wh',
     'value': 1407.9019},
    {'channelName': 'EnergySelfConsumptionTotal',
     'channelType': 'Energy',
     'unit': 'Wh',
     'value': 0.0},
    {'channelName': 'EnergyConsumptionTotal',
     'channelType': 'Energy',
     'unit': 'Wh',
     'value': 0.0},
    {'channelName': 'SavingsCO2',
     'channelType': 'CO2 savings',
     'unit': 'kg',
     'value': 0.75},
    {'channelName': 'SavingsTrees',
     'channelType': 'CO2 savings',
     'unit': 'tree',
     'value': 0.02},
    {'channelName': 'SavingsTravelCar',
     'channelType': 'CO2 savings',
     'unit': 'k

In [33]:
deviceIds = list()
for pvid, devs in devices.items():
    for dev in devs.deviceId:
        deviceIds.append((pvid, dev))
deviceIds[:5]

[('7e7aa106-19ea-4f79-ac69-e42883668ec7',
  'bb177bc0-ce82-4c49-baa1-17b30626d56b'),
 ('7e7aa106-19ea-4f79-ac69-e42883668ec7',
  '2e303432-3833-3930-3637-000000000000'),
 ('2e232019-c3b1-441c-b0ba-6c5050e8659e',
  'c30a41fc-d49d-4c5a-aa03-ac7f011d1de8'),
 ('2e232019-c3b1-441c-b0ba-6c5050e8659e',
  '3f00cf5e-1879-4624-b378-209e76206e87'),
 ('2e232019-c3b1-441c-b0ba-6c5050e8659e',
  '2e303432-3131-3439-3834-370000000000')]

In [52]:
from datetime import datetime, timedelta
now = datetime.utcnow().replace(microsecond=0)
bef = now - timedelta(days=1)
device_aggrdata = dict()
for pvid, devid in deviceIds:
    device_aggrdata[devid] = requests.get(f'{ENDPOINT}/pvsystems/{pvid}/devices/{devid}/aggrdata', headers=headers, params={
            'from': bef.strftime("%Y-%m-%d"),
            'duration': 1
        }).json()
next(iter(device_aggrdata.values()))

{'pvSystemId': '7e7aa106-19ea-4f79-ac69-e42883668ec7',
 'deviceId': 'bb177bc0-ce82-4c49-baa1-17b30626d56b',
 'data': [{'logDateTime': '2021-06-19',
   'channels': [{'channelName': 'EnergyDC1',
     'channelType': 'Energy',
     'unit': 'Wh',
     'value': 0.0},
    {'channelName': 'EnergyDC2',
     'channelType': 'Energy',
     'unit': 'Wh',
     'value': 0.0},
    {'channelName': 'EnergyExported',
     'channelType': 'Energy',
     'unit': 'Wh',
     'value': 0.0}]}],
 'links': {'first': 'https://api.solarweb.com/swqapi/pvsystems/7e7aa106-19ea-4f79-ac69-e42883668ec7/devices/bb177bc0-ce82-4c49-baa1-17b30626d56b/aggrdata?from=2021-06-19&duration=1&offset=0&limit=50',
  'prev': None,
  'self': 'https://api.solarweb.com/swqapi/pvsystems/7e7aa106-19ea-4f79-ac69-e42883668ec7/devices/bb177bc0-ce82-4c49-baa1-17b30626d56b/aggrdata?from=2021-06-19&duration=1&offset=0&limit=50',
  'next': None,
  'last': 'https://api.solarweb.com/swqapi/pvsystems/7e7aa106-19ea-4f79-ac69-e42883668ec7/devices/bb17

In [41]:
device_flows = {
    devid: requests.get(f'{ENDPOINT}/pvsystems/{pvid}/devices/{devid}/flowdata', headers=headers).json()
    for pvid, devid in deviceIds 
}
next(iter(device_flows.values()))

TypeError: 'dict' object is not callable

In [45]:
channels = dict()
for devid, dev in device_flows.items():
    if not 'pvSystemId' in dev:
        print(f"DEVID {devid} NO pvSystemId: {dev}")
        continue
    devs = devices[dev['pvSystemId']]
    dids = { row['deviceId']: row['deviceType'] for index, row in devs.iterrows() }
    if devid not in dids:
        print(f"DEVID {devid} NOT IN DEVIDS?")
        continue
    devtype = dids[devid]
    if devtype not in channels:
        channels[devtype] = set()
    else:
        if (not dev['data']) or (not 'channels' in dev['data']):
            print(f"DEVID {devid} DEVTTYPE {devtype} NO CHANNELS")
            continue
        for channel in dev['data']['channels']:
            channels[devtype].add(channel['channelName'])

print(channels)

DEVID 2e303432-3833-3930-3637-000000000000 NO pvSystemId: {'responseError': 1002, 'responseMessage': 'Requested resource not found.'}
DEVID 2e303432-3131-3439-3834-370000000000 NO pvSystemId: {'responseError': 1002, 'responseMessage': 'Requested resource not found.'}
DEVID 2e303432-3934-3236-3233-000000000000 NO pvSystemId: {'responseError': 1002, 'responseMessage': 'Requested resource not found.'}
DEVID 2e303432-3635-3036-3036-000000000000 NO pvSystemId: {'responseError': 1002, 'responseMessage': 'Requested resource not found.'}
DEVID 2e303432-3436-3731-3630-000000000000 NO pvSystemId: {'responseError': 1002, 'responseMessage': 'Requested resource not found.'}
DEVID 2e303432-3134-3533-3430-000000000000 NO pvSystemId: {'responseError': 1002, 'responseMessage': 'Requested resource not found.'}
DEVID 01f28c31-f8a0-49e2-b9a4-5838522dbe9d DEVTTYPE Inverter NO CHANNELS
DEVID 2e303432-3933-3632-3130-000000000000 NO pvSystemId: {'responseError': 1002, 'responseMessage': 'Requested resource no

In [37]:
from datetime import datetime, timedelta
now = datetime.utcnow().replace(microseconds=0)
bef = now - timedelta(seconds=3600)

histdata = {
    pvid: requests.get(f'{ENDPOINT}/pvsystems/{pvid}/histdata',
        headers=headers,
        params={
            "from": bef.isoformat(),
            "to": now.isoformat()
        }
    ).json()
    for pvid in pvsystems.pvSystemId
}
next(iter(histdata.values()))

{'pvSystemId': '7e7aa106-19ea-4f79-ac69-e42883668ec7',
 'deviceId': None,
 'data': [{'logDateTime': '2021-06-17T18:35:00Z',
   'logDuration': 299,
   'channels': [{'channelName': 'EnergySelfConsumption',
     'channelType': 'Energy',
     'unit': 'Wh',
     'value': 0.0},
    {'channelName': 'EnergyFeedIn',
     'channelType': 'Energy',
     'unit': 'Wh',
     'value': 0.0},
    {'channelName': 'EnergyBattCharge',
     'channelType': 'Energy',
     'unit': 'Wh',
     'value': 0.0},
    {'channelName': 'EnergyBattDischargeGrid',
     'channelType': 'Energy',
     'unit': 'Wh',
     'value': 0.0},
    {'channelName': 'EnergyBattDischarge',
     'channelType': 'Energy',
     'unit': 'Wh',
     'value': 0.0},
    {'channelName': 'EnergyPurchased',
     'channelType': 'Energy',
     'unit': 'Wh',
     'value': 0.0},
    {'channelName': 'EnergyBattChargeGrid',
     'channelType': 'Energy',
     'unit': 'Wh',
     'value': 0.0},
    {'channelName': 'EnergyOutput',
     'channelType': 'Energy'

In [38]:
device_histdata = {
    devid: requests.get(f'{ENDPOINT}/pvsystems/{pvid}/devices/{devid}/histdata',
        headers=headers,
        params={
            "from": bef.astimezone().replace(microsecond=0).isoformat(),
            "to": now.astimezone().replace(microsecond=0).isoformat()
        }
    ).json()
    for pvid, devid in deviceIds
}
next(iter(device_histdata.values()))

{'pvSystemId': '7e7aa106-19ea-4f79-ac69-e42883668ec7',
 'deviceId': 'bb177bc0-ce82-4c49-baa1-17b30626d56b',
 'data': [{'logDateTime': '2021-06-17T18:35:00Z',
   'logDuration': 299,
   'channels': [{'channelName': 'EnergyExported',
     'channelType': 'Energy',
     'unit': 'Wh',
     'value': 12.45},
    {'channelName': 'StandardizedPower',
     'channelType': 'Percentage',
     'unit': '%',
     'value': 1.51},
    {'channelName': 'VoltageA',
     'channelType': 'Voltage',
     'unit': 'V',
     'value': 230.0},
    {'channelName': 'VoltageB',
     'channelType': 'Voltage',
     'unit': 'V',
     'value': 227.9},
    {'channelName': 'VoltageC',
     'channelType': 'Voltage',
     'unit': 'V',
     'value': 229.2},
    {'channelName': 'CurrentA',
     'channelType': 'Current',
     'unit': 'A',
     'value': 0.23},
    {'channelName': 'CurrentB',
     'channelType': 'Current',
     'unit': 'A',
     'value': 0.23},
    {'channelName': 'CurrentC',
     'channelType': 'Current',
     'un

/home/rafa/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['pvSystemId', 'name', 'address', 'pictureURL', 'installationDate',
       'lastImport', 'meteoData', 'timeZone'],
      dtype='object')]

  exec(code_obj, self.user_global_ns, self.user_ns)


TypeError: value must be None, Series, or DataFrame

In [47]:
pvsystems.to_pickle('store/pvsystems.pickle')
for pvid, devs in devices.items():
    devs.to_pickle(f'store/{pvid}.pickle')

with open("store/all.pickle", "w+b") as outfile:
    pickle.dump({
        'flowdata': flowdata,
        'deviceIds': deviceIds,
        'device_flows': device_flows,
        'histdata': histdata,
        'device_histdata': device_histdata,
    }, outfile)

In [6]:
pvsystems = pd.read_pickle('store/pvsystems.pickle')
devices = {
    pvid: pd.read_pickle(f'store/{pvid}.pickle')
    for pvid in pvsystems.pvSystemId
}
with open("store/all.pickle", "rb") as infile:
    all = pickle.load(infile)

flowdata = all['flowdata']
deviceIds = all['deviceIds']
device_flows = all['device_flows']
histdata = all['histdata']
device_histdata = all['device_histdata']

In [7]:
pvsystems.name


0                     Beniata derecha
1            EIMA Jesuset del Miracle
2                  Cotes Baixes Norte
3                        Tomas Llacer
4     Polideportivo Francisco laporta
5               Santiago Payá Derecha
6                   Beniata izquierda
7                         San Vicente
8                     Eduardo Latorre
9               Biblioteca zona norte
10                       CEIP ROMERAL
11                Escoleta Zona Norte
12            Santiago Paya Izquierda
13                   Miguel Hernandez
14                      Cervantes CCJ
15                   Cotes Baixes Sur
16                       Huerta Mayor
Name: name, dtype: object

In [87]:
channels_device_hist = dict()
for devid, data in device_histdata.items():
    devtype = deviceTypes[devid]
    if not devtype in channels_device_hist:
        channels_device_hist[devtype] = dict()
    for itemlist in (data.get('data', None) or list()):
        for item in itemlist['channels']:
            chanelName = item['channelName']
            channels_device_hist[devtype][chanelName] = item

channels_device_hist
channels_device_hist = { key: pd.DataFrame(val.values()) for key, val in channels_device_hist.items() }
channels_device_hist.keys()

dict_keys(['Inverter', 'Datalogger', 'SmartMeter'])

In [88]:
channels_datalogger_hist = channels_device_hist['Datalogger']
channels_datalogger_hist

""


In [89]:
channels_inverter_hist = channels_device_hist['Inverter']
channels_inverter_hist

,channelName,channelType,unit,value
0,EnergyExported,Energy,Wh,5.10
1,StandardizedPower,Percentage,%,0.62
2,VoltageA,Voltage,V,231.90
3,VoltageB,Voltage,V,230.40
4,VoltageC,Voltage,V,229.10
5,CurrentA,Current,A,0.23
6,CurrentB,Current,A,0.20
7,CurrentC,Current,A,0.24
8,VoltageDC1,Voltage,V,623.30
9,CurrentDC1,Current,A,0.06


In [90]:
channels_smartmeter_hist = channels_device_hist['SmartMeter']
channels_smartmeter_hist

,channelName,channelType,unit,value
0,GridEnergyExported,Energy,Wh,0.00
1,GridEnergyImported,Energy,Wh,455.00
2,GridPowerA,Power,W,2242.17
3,GridPowerB,Power,W,1456.29
4,GridPowerC,Power,W,1761.11
5,GridApparentPowerA,Apparent Power,VA,2322.16
6,GridApparentPowerB,Apparent Power,VA,1795.28
7,GridApparentPowerC,Apparent Power,VA,1835.46
8,GridVoltageA,Voltage,V,230.43
9,GridVoltageB,Voltage,V,231.54


In [ ]:
deviceTypes = dict()
for pvid, devs in devices.items():
    for index, row in devs.iterrows():
        deviceTypes[row['deviceId']] = row['deviceType']

channels_device_flow = dict()
for devid, data in device_flows.items():
    devtype = deviceTypes[devid]
    if not devtype in channels_device_flow:
        channels_device_flow[devtype] = dict()
    for item in (data.get('data', None) or dict()).get('channels', []):
        chanelName = item['channelName']
        channels_device_flow[devtype][chanelName] = item

channels_device_flow
channels_device_flow = { key: pd.DataFrame(val.values()) for key, val in channels_device_flow.items() }
channels_device_flow.keys()

In [79]:
channels_datalogger_flow = channels_device_flow['Datalogger']
channels_datalogger_flow

""


In [73]:
channels_inverter_flow = channels_device_flow['Inverter']
channels_inverter_flow

,channelName,channelType,unit,value
0,PowerOutput,Power,W,3988.0


In [76]:
channels_meter_flow = channels_device_flow['SmartMeter']
channels_meter_flow

,channelName,channelType,unit,value
0,PowerFeedIn,Power,W,0.0


In [64]:
channels_pvsystem_flow = dict()
for pvid, data in flowdata.items():
    for item in (data.get('data', None) or dict()).get('channels', []):
        channels_pvsystem_flow[item['channelName']] = item

channels_pvsystem_flow = pd.DataFrame(channels_pvsystem_flow.values())
channels_pvsystem_flow

,channelName,channelType,unit,value
0,PowerFeedIn,Power,W,-2824.50
1,PowerLoad,Power,W,-2604.50
2,PowerBattCharge,Power,W,NaN
3,PowerPV,Power,W,5429.00
4,PowerOutput,Power,W,NaN
5,BattSOC,Percentage,%,NaN
6,RateSelfConsumption,Percentage,%,47.97
7,RateSelfSufficiency,Percentage,%,100.00


In [92]:
channels_pvsystem_hist = dict()
for pvid, data in histdata.items():
    for itemlist in (data.get('data', None) or list()):
        for item in itemlist['channels']:
            channels_pvsystem_hist[item['channelName']] = item

channels_pvsystem_hist = pd.DataFrame(channels_pvsystem_hist.values())
channels_pvsystem_hist

,channelName,channelType,unit,value
0,EnergySelfConsumption,Energy,Wh,5.1
1,EnergyFeedIn,Energy,Wh,0.0
2,EnergyBattCharge,Energy,Wh,0.0
3,EnergyBattDischargeGrid,Energy,Wh,0.0
4,EnergyBattDischarge,Energy,Wh,0.0
5,EnergyPurchased,Energy,Wh,455.0
6,EnergyBattChargeGrid,Energy,Wh,0.0
7,EnergyOutput,Energy,Wh,0.0
8,EnergyProductionTotal,Energy,Wh,5.1
9,EnergySelfConsumptionTotal,Energy,Wh,5.1
